In [ ]:
import gym

import numpy as np
from numpy import random

from collections import Counter
from functools import reduce
import matplotlib.pyplot as plt

import time
from gym.envs.toy_text import discrete

from newFrozen import FrozenLakeEnv

In [ ]:
def value_iteration(env, gamma = 1.0, eps_v = 1e-10):
    num_states = env.nS
    num_actions = env.nA
    P = env.P
    v = np.zeros(num_states)
    diff_vs = []
    v_sum = []
    iters = 0
    converged = False
    while not converged:
        v_old = np.copy(v)
        Q = np.zeros((num_states, num_actions))
        for s in range(num_states):
            for a in range(num_actions):
                for prob, s_next, reward, done in P[s][a]:
                    if not done:
                        Q[s][a] += prob * (reward + (gamma* v_old[s_next]))
                    else:
                        Q[s][a] += prob * (reward)
        v = np.max(Q,1)
        diff = np.max(np.abs(v - v_old))
        diff_vs.append(diff)
        v_sum.append(v.sum())
        if(diff < eps_v):
            converged = True
        iters += 1
    return v, iters, v_sum , diff_vs

In [ ]:
def extract_policy(env,v, gamma = 1.0):
    num_states = env.nS
    num_actions = env.nA
    P = env.P
    policy = np.zeros(num_states)
    for s in range(num_states):
        action_values = np.zeros(num_actions)
        for a in range(num_actions):
            for prob, s_next, r, done in P[s][a]:
                if not done:
                    action_values[a] += (prob * (r + gamma * v[s_next]))
                else:
                    action_values[a] += (prob * (r))
        policy[s] = np.argmax(action_values)
    return policy

def runPolicy(env, policy, useDiscount = True, gamma = 1.0):
    reward = 0
    s = env.reset()
    i = 0
    done = False
    while not done:
        s, r, done, _ = env.step(policy[s])
        if useDiscount:
            reward += (gamma**i * r)
        else:
            reward += r
        i += 1
    return reward

def evaluate_policy(env, policy, useDiscount = True, gamma = 1.0,  nSamples = 100):
    return np.mean([runPolicy(env, policy, useDiscount, gamma) for _ in range(nSamples)])

def compute_policy_v(env, policy, gamma = 1.0, eps_v = 1e-10, maxIters = 2000):
    num_states = env.nS
    num_actions = env.nA
    P = env.P
    v = np.zeros(num_states)
    iters = 0
    converged = False
    while not converged and iters < maxIters:
        v_prev = np.copy(v)
        for s in range(num_states):
            a = policy[s]
            action_value = 0
            for prob, s_next, r, done in P[s][a]:
                if not done:
                    action_value += prob * (r + gamma * v_prev[s_next])
                else:
                    action_value += prob * (r)
            v[s] = action_value
        if(np.max(np.abs(v - v_prev)) < eps_v):
            converged = True
        iters += 1
    return v, iters


def policy_iteration(env, gamma = 1.0, maxIters = 200000,  eps_v = 1e-10, nSamples = 100):
    num_states = env.nS
    num_actions = env.nA
    v_values = []
    n_iters = []
    P = env.P
    converged = False
    policy = np.random.choice(num_actions, num_states)
    i = 0
    while i < maxIters and not converged:
        policy_prev = np.copy(policy)

        v, iters = compute_policy_v(env, policy, gamma, eps_v)
        n_iters.append(iters)

        v_values.append(v.sum())

        policy = extract_policy(env, v, gamma)

        if(np.all(policy == policy_prev)):
            converged = True
        i += 1
    return policy, i, v_values, n_iters

In [ ]:
def QLearning(problem, maxIters = 200, gamma = 1.0, alpha = .01, episodes = 10000, epsilon = 1,
 epsilon_min = .1, epsilon_decay = .9):
    nState = problem.observation_space.n
    nAction = problem.action_space.n
    Q = np.random.rand(nState, nAction)
    q_scores = []
    q_iters_s = []
    q_eps = []
    cnt = Counter()
    for i in range(episodes):
        s = problem.reset()
        done = False
        Q_old = Q.copy()
        step = 0
        while not done:
            a = problem.action_space.sample()
            if random.uniform(0,1) > epsilon:            
                a = np.argmax(Q[s])
            s_, r, done, info = problem.step(a)
            q_old = Q[s,a]
            q_new = (1-alpha)*q_old + alpha*(r+gamma*np.max(Q[s_,:]))
            Q[s,a] = q_new
            s = s_
            step += 1
        epsilon = max(epsilon_min, epsilon*epsilon_decay)
        q_eps.append(epsilon)

        pol = np.argmax(Q, 1)
        cnt[str(pol)] += 1
        
        
        if(i%100 == 99):

            # most_common = cnt.most_common(3)
            # spol = reduce(lambda x,y: str(x)+str(y), pol)
            score = evaluate_policy(problem, pol)
            q_scores.append(score)
            q_iters_s.append(i)
            
            # print("iteration: {}\t epsilon: {}\t policy: {}\t score: {} \t most common: {}".format(i,epsilon,spol,score,[x[1] for x in most_common]))
        

    

    # plt.savefig(title + "/Iterations per policy")
    # plt.close()
    return np.argmax(Q, 1), episodes, q_iters_s ,q_scores, q_eps

In [ ]:
frozen_4 = FrozenLakeEnv(map_name="4x4")
frozen_20 = FrozenLakeEnv(map_name="20x20")

In [ ]:
fl_envs = [frozen_4]
decay = 0.99
title = "test"

In [ ]:
for problem in fl_envs:
    if hasattr(problem, 'env'):
        env = problem.env
    else:
        env = problem
    print("XXXXXX" , str(env), "XXXXXX")
    print("states: {}\n actions: {}".format(env.nS, env.nA))

    
    t0 = time.perf_counter()
    v_value, i_value, summed, diff = value_iteration(env, 1.0)

    plt.plot(diff)
    plt.title(title + " Diff in summed V- VI")
    plt.xlabel("Iterations")
    plt.ylabel("Diff is summed V")
#     plt.savefig( title + "/v_diff")
#     plt.close()
    plt.show()
    plt.close()


    plt.plot(summed)
    plt.title(title + " state Values -VI")
    plt.xlabel("Iterations")
    plt.ylabel("sum of values per state")
#     plt.savefig(title + "/v_summed")
#     plt.close()
    plt.show()
    plt.close()


    tf = time.perf_counter()
    t_value = tf-t0
    # get policy
    p_star_value = extract_policy(env, v_value)
    score_value = evaluate_policy(env, p_star_value, False)
    #####
    print("VI:\n\ttime: {}\n\titers: {}\n\tscore: {}".format(t_value, i_value,score_value))
    ###########


    
    t0 = time.perf_counter()
    p_star_policy, i_policy, p_values, n_iters = policy_iteration(env)
    # print(p_star_policy)
    plt.plot(range(1,i_policy+1),p_values)
    print("highest PI V: ", p_values[-1])
    plt.title(title + " Policy values - PI")
    plt.xlabel("policiy number")
    plt.ylabel("summed value for policy")
#     plt.savefig(title + "/P_summed")
    plt.show()
    plt.close()

    plt.plot(range(1,i_policy+1),n_iters)
    # print(n_iters)
    plt.title(title + " iterations per V of Policy - PI")
    plt.xlabel("policiy number")
    plt.ylabel("# of Iterations")
#     plt.savefig(title + "/Iterations per policy")
#     plt.close()
    plt.show()
    plt.close()


    tf = time.perf_counter()
    t_policy = tf-t0
    # derive value matrix
    v_policy = compute_policy_v(env, p_star_policy)
    # v_policys = [compute_policy_v(env, i) for  ]
    score_policy = evaluate_policy(env, p_star_policy, False)
    print("Policy Iteration:\n\ttime: {}\n\titers: {}\n\tscore: {}".format(t_policy, i_policy,score_policy))
    #####    

    print("I policy the same: {}".format(np.all(p_star_value == p_star_policy)))



        # do Q-learning

    t0 = time.perf_counter()
    p_star_Q, i_Q, q_iters_s, q_scores_iter, q_eps = QLearning(problem, epsilon_decay = decay)
    tf = time.perf_counter()
    t_Q = tf-t0
    score_Q = evaluate_policy(env, p_star_Q, False)

    plt.plot(q_iters_s,q_scores_iter)
    # print(n_iters)
    plt.title(title + "Qlearning scores vs Iterations. decay factor " + str(decay))
    plt.xlabel("Iterations")
    plt.ylabel("scores")
    name = title + "/" + str(decay) +" Qlearning scores.png"
#     plt.savefig(name)
    plt.show()
    plt.close()

    plt.plot(q_eps)
    # print(n_iters)
    plt.title(title + "QL Decay: epsilon vs Iterations. decay factor " + str(decay))
    plt.xlabel("Iterations")
    plt.ylabel("epsilon")
    name1 = title + "/" + str(decay) +" epsilon_decay.png"
#     plt.savefig(name1)
    plt.show()
    plt.close()


    print("QL:\n\ttime: {}\n\titers: {}\n\tscore: {}".format(t_Q, i_Q,score_Q))
    print("")